In [1]:
import pandas as pd
import numpy as np

In [67]:
df = pd.read_csv("../data/data_imputed_2.csv")

In [68]:
df.head()

,timestamp_local,temp,city_name,country_code,aqi,co,no2,o3,pm10,pm25,so2
0,01/02/2022 0:00,12.6,Gujrāt,PK,385,1339.8,76.0,10.7,491.7,347.67,238.0
1,01/02/2022 1:00,11.5,Gujrāt,PK,404,1437.6,76.0,9.3,508.3,359.33,268.0
2,01/02/2022 2:00,11.9,Gujrāt,PK,421,1535.5,76.0,8.0,525.0,371.00,298.0
3,01/02/2022 3:00,12.2,Gujrāt,PK,425,1659.0,68.3,5.3,529.3,374.00,275.7
4,01/02/2022 4:00,11.9,Gujrāt,PK,430,1782.5,60.7,2.7,533.7,377.00,253.3


In [69]:
import numpy as np
import matplotlib.pyplot as plt

def create_series(df, xcol, datecol):
    # Create a dataframe with the features and the date time as the index
    features_considered = [xcol]
    features = df[features_considered]
    features.index = df[datecol]
    # features.head()
    # features.plot(subplots=True)
    return features


# X is the series to test
# log_x asks whether to log X prior to testing or not
def stationarity_test(X, log_x = "Y", return_p = False, print_res = True):
    
    # If X isn't logged, we need to log it for better results
    if log_x == "Y":
        X = np.log(X[X>0])
    
    # Once we have the series as needed we can do the ADF test
    from statsmodels.tsa.stattools import adfuller
    dickey_fuller = adfuller(X)
    
    if print_res:
    # If ADF statistic is < our 1% critical value (sig level) we can conclude it's not a fluke (ie low P val / reject H(0))
        print('ADF Stat is: {}.'.format(dickey_fuller[0]))
        # A lower p val means we can reject the H(0) that our data is NOT stationary
        print('P Val is: {}.'.format(dickey_fuller[1]))
        print('Critical Values (Significance Levels): ')
        for key,val in dickey_fuller[4].items():
            print(key,":",round(val,3))
            
    if return_p:
        return dickey_fuller[1]
    
# Differencing the data    
def difference(X):
    diff = X.diff()
    plt.plot(diff)
    plt.show()
    return diff

In [70]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

In [71]:
temp = df[['aqi']]

In [72]:
temp

,aqi
0,385
1,404
2,421
3,425
4,430
...,...
19701,89
19702,102
19703,107
19704,112


In [73]:
from sklearn.preprocessing import MinMaxScaler
values = temp.values

values = values.astype('float32')

scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

In [74]:
scaled.shape

(19706, 1)

In [75]:
scaled = scaled.reshape((19706,))

In [76]:
scaled.shape

(19706,)

In [77]:
scaled

array([0.75374734, 0.7944325 , 0.8308351 , ..., 0.15845825, 0.16916488,
       0.18201286], dtype=float32)

In [78]:
def df_to_X_y(df, window_size, forecast_length):
    # df_as_np = df.to_numpy()
    X = []
    y = []
    
    for i in range(len(df) - window_size - forecast_length):
        row_x = [[a] for a in df[i:i+window_size]]
        
        X.append(row_x)
        # row_y = df_as_np[i+window_size]
        row_y = [b for b in df[i+window_size:i+window_size+forecast_length]]
        
        y.append(row_y)
        
    return np.array(X), np.array(y)

In [90]:
WINDOW_WIDTH = 168
FORECAST_LENGTH = 1
X, y = df_to_X_y(scaled, WINDOW_WIDTH, FORECAST_LENGTH)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

X_max = X_train.max()
X_min = X_train.min()

In [91]:
X.shape

(19537, 168, 1)

In [92]:
batchSize = 16
from keras.models import Sequential, save_model, load_model
from keras.layers import *
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError

In [94]:
normalizer = Normalization()
normalizer.adapt(X_train)
rlrop = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)

model = Sequential()
# 1D Convolutional layers
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=X_train.shape[1:]))
# model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
# model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
# Flatten the output of the Conv layers
model.add(Flatten())
# model.add(LSTM(input_shape = X_train.shape[1:], units=86, return_sequences=True))
# model.add(LSTM(units=64, return_sequences=True))
# model.add(GRU(units=86, return_sequences=True))
# model.add(LSTM(units=128, return_sequences=True))
# model.add(GRU(units=100, return_sequences=True))
# model.add(LSTM(units=100, return_sequences=False))
# model.add(Dense(units=100,  activation='relu'))
# model.add(Dense(8, activation='relu'))

model.add(Dense(units=FORECAST_LENGTH, activation='linear'))

model.summary()

model.compile(optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()] , loss=MeanSquaredError())

history = model.fit(X_train, y_train, validation_data=(X_val,y_val), epochs=50, callbacks=[rlrop])

# score = model.evaluate(X_test, y_test, verbose = 1)

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_4 (Conv1D)                    │ (None, 166, 64)             │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_4 (Flatten)                  │ (None, 10624)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 1)                   │          10,625 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 10,881 (42.50 KB)

 Trainable params: 10,881 (42.50 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0104 - root_mean_squared_error: 0.0954 - val_loss: 0.0021 - val_root_mean_squared_error: 0.0459 - learning_rate: 0.0010
Epoch 2/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0022 - root_mean_squared_error: 0.0472 - val_loss: 0.0018 - val_root_mean_squared_error: 0.0426 - learning_rate: 0.0010
Epoch 3/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.0018 - root_mean_squared_error: 0.0421 - val_loss: 0.0022 - val_root_mean_squared_error: 0.0465 - learning_rate: 0.0010
Epoch 4/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0016 - root_mean_squared_error: 0.0404 - val_loss: 0.0013 - val_root_mean_squared_error: 0.0365 - learning_rate: 0.0010
Epoch 5/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0016 - root_mean_squared_error: 0.0399 - val_loss: 0.0014 - val_root_mean_squared_error: 0.0373 - learning_rate: 0.0010
Epoch 6/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0015 - root_mean_squared_err

In [95]:
forecast = model.predict(X_test)

123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [96]:
poll = np.array(df["aqi"])

meanop = poll.mean()
stdop = poll.std()

y_test_true = y_test*stdop + meanop
testPredict = forecast*stdop + meanop

In [97]:
from sklearn.metrics import mean_squared_error

In [98]:
rmse = np.sqrt(mean_squared_error(y_test_true, testPredict))
print("Test RMSE ="  ,rmse)

Test RMSE = 2.2266464


In [99]:
forecast

array([[0.3182087 ],
       [0.12527588],
       [0.17633861],
       ...,
       [0.13315398],
       [0.21843788],
       [0.42623976]], dtype=float32)

In [100]:
testPredict

array([[174.06168],
       [162.31024],
       [165.42044],
       ...,
       [162.79008],
       [167.9847 ],
       [180.6418 ]], dtype=float32)

In [101]:
y_test_true

array([[172.54828],
       [160.80983],
       [166.28777],
       ...,
       [163.41837],
       [165.37479],
       [182.3303 ]], dtype=float32)

In [102]:
# Use the last available data point from your dataset as the initial input
last_input = X_test[-168:]

# Make prediction
forecast = model.predict(last_input)

# Inverse transform forecasted value
forecast = forecast.ravel() * stdop + meanop
print(forecast)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
[169.91458 187.40926 160.5014  162.76108 168.45111 171.68578 162.11377
 185.31924 174.57965 171.76117 160.30963 164.90567 169.68782 165.88435
 162.34909 172.54448 169.30971 166.24742 192.7403  178.83891 164.91603
 165.73605 165.58168 176.06604 166.37267 178.01857 174.65485 160.69594
 178.18405 173.64241 181.75803 165.87349 171.71962 175.6577  165.68875
 169.26404 167.57684 165.34644 183.09581 172.35182 163.9808  174.7425
 158.02592 168.70834 173.88136 163.20085 173.23633 174.35324 161.39038
 207.6283  169.04935 162.1792  163.14197 162.31961 172.23    179.2592
 181.43954 173.33018 173.76929 171.84698 170.96667 172.25061 177.15411
 165.72638 180.82788 176.58582 175.53667 176.25334 164.1777  178.52863
 176.33417 168.77693 170.25299 159.5578  162.41032 160.71149 165.74657
 182.75111 162.07297 167.07416 170.1425  162.7813  169.264   173.29195
 167.2552  177.2388  174.50835 166.80717 171.14905 167.44427 173.59058
 171.51314 176.0796  173.7372  161.64258 

In [104]:
y_test[-168:].ravel() * stdop + meanop

array([170.07016, 187.15611, 161.20111, 160.94025, 163.80965, 171.89613,
       161.85324, 190.28638, 173.98297, 172.80913, 159.7664 , 164.59221,
       169.67888, 162.24452, 164.98349, 172.6787 , 169.02673, 166.54863,
       190.9385 , 180.3739 , 165.89648, 166.4182 , 166.28777, 178.15665,
       166.4182 , 178.4175 , 172.80913, 159.7664 , 179.06964, 174.37425,
       181.93903, 163.67923, 173.46127, 174.24384, 166.93991, 169.67888,
       168.11375, 165.76607, 184.93886, 172.54828, 164.33136, 173.46127,
       158.46214, 168.50504, 177.5045 , 162.89667, 170.46144, 173.46127,
       161.20111, 207.89404, 170.07016, 161.72281, 163.15752, 164.98349,
       186.1127 , 179.20006, 183.50415, 173.06998, 172.93956, 172.157  ,
       171.37444, 172.54828, 177.11322, 164.59221, 184.93886, 177.11322,
       171.63528, 171.89613, 163.94008, 178.02621, 176.85237, 167.85289,
       170.72229, 159.37512, 161.85324, 161.20111, 166.67905, 185.98227,
       162.24452, 167.72247, 168.63545, 162.37495, 